In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [2]:
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")

In [3]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [4]:
# Set the title of the dashboard
app.title = "Automobile Statistics Dashboard"

In [5]:
# Create the dropdown menu options
dropdown_options = [
    {"label": "Yearly Statistics", "value": "Yearly Statistics"},
    {"label": "Recession Period Statistics", "value": "Recession Period Statistics"}]

In [6]:
# List of years
year_list = [i for i in range(1980, 2024, 1)]

In [7]:
# Create the layout of the app
app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard",style={"textAlign": "center", "color": "#503D36", "font-size": 24}),  # May include style for title
    html.Div([html.Label("Select Statistics:"), dcc.Dropdown(id="dropdown-statistics", options=[{"label": "Yearly Statistics", "value": "Yearly Statistics"},{"label": "Recession Period Statistics","value": "Recession Period Statistics"}],value="Select Statistics",placeholder="Select a report type")]),
    html.Div(dcc.Dropdown(id="select-year", options=[{"label": i, "value": i} for i in year_list], value="Select Year")),
    html.Div([html.Div(id="output-container", className="chart-grid", style={"flex"})])])

In [8]:
@app.callback(
    Output(component_id="select-year", component_property="disabled"),
    Input(component_id="dropdown-statistics", component_property="value"))
def update_input_container(selected_statistics):
    if selected_statistics == "Yearly Statistics":
        return False
    else:
        return True

In [9]:
@app.callback(
    Output(component_id="output-container", component_property="children"),
    [Input(component_id="dropdown-statistics", component_property="value"),Input(component_id="select-year", component_property="value")])

def update_output_container(selected_statistics, input_year):
    if selected_statistics == "Recession Period Statistics":
        recession_data = data[data["Recession"] == 1]
        
        yearly_rec = recession_data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(
                yearly_rec, 
                x="Year", 
                y="Automobile_Sales", 
                title="Average Automobile Sales fluctuation over Recession Period"))

        average_sales = recession_data.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(
                average_sales,
                x="Vehicle_Type",
                y="Automobile_Sales",
                title="Average Number of Vehicles sold by Vehicle Type"))

        exp_rec = recession_data.groupby("Vehicle_Type")["Advertising_Expenditure"].sum().reset_index()

        R_chart3 = dcc.Graph(
            figure=px.pie(
                exp_rec,
                names="Vehicle_Type",
                values="Advertising_Expenditure",
                title="Total Expenditure Share by Vehicle Type during Recessions"))

        unemp_data = recession_data.groupby(["Vehicle_Type", "unemployment_rate"])["Automobile_Sales"].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(
                unemp_data,
                x="unemployment_rate",
                y="Automobile_Sales",
                color="Vehicle_Type",
                labels={"unemployment_rate": "Unemployment Rate","Automobile_Sales": "Average Automobile Sales"},
                title="Effect of Unemployment Rate on Sales of various Vehicle Types"))

        return [html.Div(className="chart-item", children=[html.Div(children=R_chart1), html.Div(children=R_chart2)], style={"display": "flex"}),
                html.Div(className="chart-item", children=[html.Div(children=R_chart3), html.Div(children=R_chart4)], style={"display": "flex"})]

    elif input_year and selected_statistics == "Yearly Statistics":
        yearly_data = data[data["Year"] == input_year]

        yas = data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(
                yas, x="Year", y="Automobile_Sales", title="Yearly Automobile sales"))

        Y_chart2 = dcc.Graph(
            figure=px.line(
                yearly_data,
                x="Month",
                y="Automobile_Sales",
                title="Total Monthly Automobile Sales"))

        avr_vdata = yearly_data.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(
                avr_vdata,
                x="Vehicle_Type",
                y="Automobile_Sales",
                title="Average Vehicles Sold by Vehicle Type in the year {}".format(input_year)))

        exp_data = yearly_data.groupby("Vehicle_Type")["Advertising_Expenditure"].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(
                exp_data,
                values="Advertising_Expenditure",
                names="Vehicle_Type",
                title="Total Advertisement Expenditure for each Vehicle"))

        return [html.Div(className="chart-item", children=[html.Div(Y_chart1), html.Div(Y_chart2)], style={"display": "flex"}),
                html.Div(className="chart-item", children=[html.Div(Y_chart3), html.Div(Y_chart4)], style={"display": "flex"})]

    else:

        return None



In [10]:
if __name__ == "__main__":
    app.run_server(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File ~\anaconda3\envs\FirstEnvironment\Lib\site-packages\flask\app.py:867, in Flask.full_dispatch_request(self=<Flask '__main__'>)
    865     rv = self.preprocess_request()
    866     if rv is None:
--> 867         rv = self.dispatch_request()
        self = <Flask '__main__'>
        rv = None
    868 except Exception as e:
    869     rv = self.handle_user_exception(e)

File ~\anaconda3\envs\FirstEnvironment\Lib\site-packages\flask\app.py:852, in Flask.dispatch_request(self=<Flask '__main__'>)
    850 # otherwise dispatch to the handler for that endpoint
    851 view_args: dict[str, t.Any] = req.view_args  # type: ignore[assignment]
--> 852 return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
        rule = <Rule '/_dash-layout' (OPTIONS, HEAD, GET) -> /_dash-layout>
        view_args = {}
        self.view_funct